In [1]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [2]:
# fetch the data file, create the dataframe.
df = pd.read_csv("16P.csv", encoding = "windows 1252")

In [3]:
df

,Response Id,You regularly make new friends.,You spend a lot of your free time exploring various random topics that pique your interest,Seeing other people cry can easily make you feel like you want to cry too,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",You prefer to completely finish one project before starting another.,You are very sentimental.,You like to use organizing tools like schedules and lists.,...,You believe that pondering abstract philosophical questions is a waste of time.,"You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.",You know at first glance how someone is feeling.,You often feel overwhelmed.,You complete things methodically without skipping over any steps.,You are very intrigued by things labeled as controversial.,You would pass along a good opportunity if you thought someone else needed it more.,You struggle with deadlines.,You feel confident that things will work out for you.,Personality
0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,-1,0,0,0,0,0,ENFP
1,1,0,0,-2,-3,-1,2,-2,0,3,...,0,-2,0,2,0,-1,-1,-1,3,ISFP
2,2,0,0,2,0,-1,2,0,0,1,...,0,2,0,2,-1,0,1,2,1,INFJ
3,3,0,-1,3,-1,0,0,-2,0,-2,...,0,0,-1,-1,0,1,0,-2,-1,ISTP
4,4,0,0,-1,0,2,-1,-2,0,1,...,0,1,0,2,0,1,-1,2,-1,ENFJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59994,59994,0,-1,0,-3,-2,2,1,0,1,...,0,2,0,1,0,0,2,3,3,INFJ
59995,59995,0,0,-2,1,3,0,-3,0,2,...,0,1,0,0,0,0,2,0,-2,ESTP
59996,59996,0,0,1,0,-2,-2,0,0,-1,...,-1,2,0,-3,0,1,-1,0,-1,ISTP
59997,59997,0,0,1,-1,-2,-1,2,0,1,...,0,2,0,2,0,0,0,1,0,ISTJ


In [4]:
# drop the "Response Id" column because it has no effect on the personality type.
df.drop(["Response Id"], axis=1, inplace=True)

In [5]:
# assign each personality type to a number.
personality_codes = {"ESTJ":0,"ENTJ":1,"ESFJ":2,"ENFJ":3,"ISTJ":4,"ISFJ":5,"INTJ":6,"INFJ":7,"ESTP":8,"ESFP":9,"ENTP":10,"ENFP":11,"ISTP":12,"ISFP":13,"INTP":14,"INFP":15}

In [6]:
# encode the "Personality" column to consist of personality codes instead of personality types.
df.replace({"Personality": personality_codes}, inplace = True)

In [7]:
# get two copies of df, one of them with feature normalization, one of them without.
df_feature_normalized = df.copy()   # with normalization
df_not_normalized = df.copy()   # without normalization

# apply min-max normalization on the dataframe.
for feature in df_feature_normalized:
    if feature == "Personality":
        break
    min_value = df_feature_normalized[feature].min()
    max_value = df_feature_normalized[feature].max()
    df_feature_normalized[feature] = [(value - min_value)/(max_value - min_value) for value in df[feature]] # apply the formula
df_feature_normalized

,You regularly make new friends.,You spend a lot of your free time exploring various random topics that pique your interest,Seeing other people cry can easily make you feel like you want to cry too,You often make a backup plan for a backup plan.,"You usually stay calm, even under a lot of pressure","At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know",You prefer to completely finish one project before starting another.,You are very sentimental.,You like to use organizing tools like schedules and lists.,Even a small mistake can cause you to doubt your overall abilities and knowledge.,...,You believe that pondering abstract philosophical questions is a waste of time.,"You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.",You know at first glance how someone is feeling.,You often feel overwhelmed.,You complete things methodically without skipping over any steps.,You are very intrigued by things labeled as controversial.,You would pass along a good opportunity if you thought someone else needed it more.,You struggle with deadlines.,You feel confident that things will work out for you.,Personality
0,0.666667,0.5,0.500000,0.500000,0.500000,0.666667,0.666667,0.333333,0.500000,0.5,...,0.5,0.500000,0.5,0.333333,0.5,0.666667,0.500000,0.500000,0.500000,11
1,0.666667,0.5,0.166667,0.000000,0.333333,0.833333,0.166667,0.333333,1.000000,0.5,...,0.5,0.166667,0.5,0.833333,0.5,0.333333,0.333333,0.333333,1.000000,13
2,0.666667,0.5,0.833333,0.500000,0.333333,0.833333,0.500000,0.333333,0.666667,0.5,...,0.5,0.833333,0.5,0.833333,0.0,0.666667,0.666667,0.833333,0.666667,7
3,0.666667,0.0,1.000000,0.333333,0.500000,0.500000,0.166667,0.333333,0.166667,0.5,...,0.5,0.500000,0.0,0.333333,0.5,1.000000,0.500000,0.166667,0.333333,12
4,0.666667,0.5,0.333333,0.500000,0.833333,0.333333,0.166667,0.333333,0.666667,0.5,...,0.5,0.666667,0.5,0.833333,0.5,1.000000,0.333333,0.833333,0.333333,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59994,0.666667,0.0,0.500000,0.000000,0.166667,0.833333,0.666667,0.333333,0.666667,0.5,...,0.5,0.833333,0.5,0.666667,0.5,0.666667,0.833333,1.000000,1.000000,7
59995,0.666667,0.5,0.166667,0.666667,1.000000,0.500000,0.000000,0.333333,0.833333,0.5,...,0.5,0.666667,0.5,0.500000,0.5,0.666667,0.833333,0.500000,0.166667,8
59996,0.666667,0.5,0.666667,0.500000,0.166667,0.166667,0.500000,0.333333,0.333333,0.5,...,0.0,0.833333,0.5,0.000000,0.5,1.000000,0.333333,0.500000,0.333333,12
59997,0.666667,0.5,0.666667,0.333333,0.166667,0.333333,0.833333,0.333333,0.666667,0.5,...,0.5,0.833333,0.5,0.833333,0.5,0.666667,0.500000,0.666667,0.500000,4


In [8]:
import numpy as np
import math

In [9]:
# convert both DataFrames into numpy arrays.
df_not_normalized = df_not_normalized.to_numpy()
df_feature_normalized = df_feature_normalized.to_numpy()

In [10]:
# calculate the accuracy, precision and recall with the macro average method.
def measure_performance(k, first_index, last_index, tp, tn, fp, fn):
    '''
    Measures the performance of the model for a k value by calculating the accuracy, precision and recall.
    
    Parameters k, first_index and last_index are the k value, first index and last index of the test group.
    These are only used for displaying.
    
    Parameters tp, tn, fp, fn are dictionaries that store tp, tn, fp, fn for each class.
    
    Doesn't return anything, only calculates the accuracy, precision and recall and prints them.
    '''
    total_precision, total_recall, total_accuracy, total_tp, total_fp, total_fn, total_tn = (0,) * 7
    for p_class in range(0,16):
        class_tp, class_tn, class_fp, class_fn = tp[p_class], tn[p_class], fp[p_class], fn[p_class]
        total_precision += class_tp / (class_tp + class_fp)
        total_recall += class_tp / (class_tp + class_fn)
        total_accuracy += (class_tp + class_tn) / (class_tp + class_tn + class_fp + class_fn)
    macro_accuracy, macro_precision, macro_recall = total_accuracy / 16, total_precision / 16, total_recall / 16
    print("Test Group: Respondants of ID's %d to %d, K value = %d\nAccuracy:\t%.12f\nPrecision:\t%.12f\nRecall:\t\t%.12f\n-----------------------------------"% (first_index, last_index, k, macro_accuracy, macro_precision, macro_recall))

In [11]:
# increment the tp, tn, fp, fn values based on the prediction.
def determine_prediction_values(tp, tn, fp, fn, predicted, actual):
    '''
    Calculates the tp, tn, fp and fn values of each class by comparing
    the prediction to the actual class.
    
    Parameters tp, tn, fp, fn are dictionaries defined in the knn function
    that store each class' respective prediction value. e.g. tp stores tp values
    of each class.
    
    Parameters predicted and actual are predicted class and real/actual class of the test
    case
    
    Doesn't return anything, only mutates the tp, tn, fp, fn dictionaries
    
    '''
    for p_class in range(0,16):
        if predicted == p_class and actual == p_class:
            tp[p_class] += 1
        elif predicted == p_class and actual != p_class:
            fp[p_class] += 1
        elif predicted != p_class and actual == p_class:
            fn[p_class] += 1
        else:
            tn[p_class] += 1

In [12]:
# defining the knn algorithm
def knn(train_array, test_array, first_index, last_index):
    '''
    Applies the knn algorithm for a test set and a training set. After performing the algorithm for each k value,
    measures the performance and outputs it.
    
    Parameters train_array and test_array are train and test sets in the form of numpy arrays.
    
    Parameters first_index and last_index are only for the measure_performance function.
    
    Doesn't return anything, only performs the algorithm and measures performance.
    '''
    
    train_predictor, test_predictor = train_array[:, :-1], test_array[:, :-1]
    train_target, test_target = train_array[:, -1].astype(int), test_array[:, -1]
    
    test_rows = test_predictor.shape[0]
    train_rows = train_predictor.shape[0]
    
    # determines the distances using matrix multiplication
    test_cases = ((test_predictor ** 2).sum(axis = 1)).reshape((test_rows, 1)) * np.ones([1, train_rows])
    train_cases = (train_predictor ** 2).sum(axis = 1) * np.ones([test_rows,1])
    distances = (test_cases + train_cases -2 * test_predictor.dot(train_predictor.T)) ** (1/2)
    
    shortest_distance_indeces = np.argsort(distances)
    
    for k in range(1, 10, 2):
        
        test_index = 0
        
        tp, tn, fp, fn = {i:0 for i in range(0,16)}, {i:0 for i in range(0,16)}, {i:0 for i in range(0,16)}, {i:0 for i in range(0,16)}
        
        for index_array in shortest_distance_indeces:
            
            correct_personality = int(test_target[test_index])
            
            nearest_personalities = train_target[index_array[:k]]
            
            freq_array = np.bincount(nearest_personalities)             # get the count of all elements in nearest personalities
            
            if np.count_nonzero(freq_array == max(freq_array)) == 1:    # if there is only 1 dominant personality
                
                predicted_personality = freq_array.argmax()
                
            else:

                predicted_personality = "nan"
                
            test_index += 1 
            
            determine_prediction_values(tp,tn,fp,fn,predicted_personality,correct_personality)
            
        measure_performance(k, first_index, last_index, tp,tn,fp,fn)

In [13]:
# implement k-fold cross-validation method
def five_fold_cross_validation(data_array):
    '''
    Starts testing the model by cutting the whole data set into 5 groups and taking each group and the remaining groups
    (for each group) and calling the knn function.
    
    Parameter data_array is our data set
    
    Doesn't return anything, only used for starting the model.
    
    '''
    for last_index in range(12000,60001,12000):
        first_index = last_index - 12000
        test_array = data_array[first_index:last_index, :]
        train_array = np.concatenate((data_array[:first_index, :], data_array[last_index: , :]), axis=0)
        knn(train_array, test_array, first_index, last_index-1)

In [14]:
# testing the model with non-normalized data set.
five_fold_cross_validation(df_not_normalized)

Test Group: Respondants of ID's 0 to 11999, K value = 1
Accuracy:	0.997333333333
Precision:	0.978665079094
Recall:		0.978613092202
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 3
Accuracy:	0.998609375000
Precision:	0.989377340278
Recall:		0.988376811172
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 5
Accuracy:	0.998666666667
Precision:	0.989458931497
Recall:		0.989209875592
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 7
Accuracy:	0.998697916667
Precision:	0.989782925136
Recall:		0.989377644441
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 9
Accuracy:	0.998703125000
Precision:	0.989699721736
Recall:		0.989544245347
-----------------------------------
Test Group: Respondants of ID's 12000 to 23999, K value = 1
Accuracy:	0.997104166667
Precision:	0.976859625712
Recall:		0.976824351314
------------------------------

In [15]:
# testing the model with feature normalized data set.
five_fold_cross_validation(df_feature_normalized)

Test Group: Respondants of ID's 0 to 11999, K value = 1
Accuracy:	0.996708333333
Precision:	0.973713743117
Recall:		0.973683907618
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 3
Accuracy:	0.998531250000
Precision:	0.989022815504
Recall:		0.987467659805
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 5
Accuracy:	0.998598958333
Precision:	0.989100112831
Recall:		0.988468857266
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 7
Accuracy:	0.998666666667
Precision:	0.989599536957
Recall:		0.989048898636
-----------------------------------
Test Group: Respondants of ID's 0 to 11999, K value = 9
Accuracy:	0.998687500000
Precision:	0.989616704531
Recall:		0.989377821909
-----------------------------------
Test Group: Respondants of ID's 12000 to 23999, K value = 1
Accuracy:	0.996697916667
Precision:	0.973677207613
Recall:		0.973548581425
------------------------------

Error Analysis:

a) Performance Analysis
- Neighbor Number (k) : For all test/training groups, we can observe that as k increases from 1 to 9, the accuracy, precision and recall of the model increase too. In some scenarios, these values or only some of them decrease as k goes from 7 to 9, but in the majority, they all increase.
- Normalization : Performing feature normalization on the data gave worse results in performance compared to the performance on the original data. Additionalliy, in some scenarios, the accuracy stayed the same. This may be because of the data set not including any outlier cases and only including values that are close to each other.
- kNN Algorithm : The kNN algorithm is very slow for this particular data set because there are a lot of respondants and a lot of questions, vectors of length 60 are created and their distance is calculated. However, the accuracy of this algorithm for this data set is very high.
- 5-Fold : Testing the model with lots of data and lots of times helped ensure the correctness of the accuracy, precision and recall as they didn't differ that much among testing groups. Also, every row in our data set was tested and trained with because of k-Fold cross validation.
- Accuracy/Precision/Recall : These values were calculated with the macro average method. They were dependent on the k value, test/train groups and normalizaton. They usually decreased or increased at the same time.

b) Misclassified Test Cases
- The algorithm sometimes cannot find a dominant class between the nearest neignbors(if k > 1), so it cannot predict anything. Also, in case of some personalities being very close to each other, the model may predict the wrong class. This may be due to the value of k or the kNN algorithm itself as it is just an algorithm that guesses depending on the distance and not on the scientific states about personalities.

An example: Respondant 4375.
Respondant 4375 has the ISTP personality type ("12"). If we calculate the nearest neighbors for each k value:

k = 1

[12]  => k nearest neigbors

Predicts correctly! (12)

k = 3

[12 12 14]

Again, predicts correctly! (12)

k = 5

[12 12 14  6  6]

Fails to predict the class. (tie between 6 and 12)

k = 7

[12 12 14  6  6  6 12]

Fails again at another k value.

- Also important to note that if a test case fails at a k value (just like the example above), I have observed that it has a considerable chance of failing at another k value. This shows that there is no clear indicator of the class for this test case and that the kNN algorithm may not be the best way to classify this particular test case. 

Here is an example of another test case (Respondant 12642, personality type 0)

k = 5

[ 3  0 11  3  0]

Fails at k = 5 first.

k = 7

[ 3  0 11  3  0  0  3]

Fails again at another k value.